# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [59]:
import pandas as pd
import numpy as np

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

transitions.head(), rewards.head()

(       action      state  next_state  probability
 0  do-nothing  exposed-1   exposed-2          0.8
 1  do-nothing  exposed-1   recovered          0.2
 2  do-nothing  exposed-2   exposed-3          0.8
 3  do-nothing  exposed-2   recovered          0.2
 4  do-nothing  exposed-3  symptoms-1          0.8,
        action       state  reward
 0  do-nothing   exposed-1     0.0
 1  do-nothing   exposed-2     0.0
 2  do-nothing   exposed-3     0.0
 3  do-nothing  symptoms-1    -0.5
 4  do-nothing  symptoms-2    -1.0)

In [60]:
# YOUR CHANGES HERE

action = "do-nothing"
P = transitions[transitions["action"] == action]
R = rewards[rewards["action"] == action].set_index("state")["reward"].to_dict()

states = sorted(set(P["state"]) | set(P["next_state"]))

n = len(states)
P_matrix = np.zeros((n, n))

for _, row in P.iterrows():
    i = states.index(row["state"])
    j = states.index(row["next_state"])
    P_matrix[i, j] = row["probability"]

R_vec = np.array([R.get(s, 0.0) for s in states])

gamma = 1
V = np.zeros(n)
threshold = 1e-8

while True:
    V_new = R_vec + gamma * P_matrix.dot(V)
    if np.max(np.abs(V_new - V)) < threshold:
        break
    V = V_new

discomfort = -V


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [61]:
# YOUR CHANGES HERE

result = pd.DataFrame({"state": states, "expected_discomfort": discomfort})
result.to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)
result.head()

,state,expected_discomfort
0,exposed-1,3.413333
1,exposed-2,4.266667
2,exposed-3,5.333333
3,recovered,-0.000000
4,symptoms-1,6.666667


Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [62]:
# YOUR CHANGES HERE

states = sorted(set(transitions["state"]) | set(transitions["next_state"]) | set(rewards["state"]))
actions = sorted(set(transitions["action"]) | set(rewards["action"]))

P = {}
for s in states:
    for a in actions:
        mask = (transitions["state"] == s) & (transitions["action"] == a)
        if mask.any():
            P[(s, a)] = list(zip(transitions.loc[mask, "next_state"], transitions.loc[mask, "probability"]))

# Reward dictionary
R = {(row["state"], row["action"]): row["reward"] for _, row in rewards.iterrows()}


gamma = 1
V = {s: 0.0 for s in states}
threshold = 1e-8

def q_value(state, action, V_current):
    r = R.get((state, action), 0.0)
    future = 0.0
    if (state, action) in P:
        for s2, p in P[(state, action)]:
            future += p * V_current[s2]
    return r + gamma * future

# Value iteration
while True:
    delta = 0.0
    V_new = {}
    for s in states:
        q_vals = [q_value(s, a, V) for a in actions if (s, a) in R or (s, a) in P]
        if q_vals:
            V_new[s] = max(q_vals)
        else:
            V_new[s] = V[s]
        delta = max(delta, abs(V_new[s] - V[s]))
    V = V_new
    if delta < threshold:
        break

# Optimal policy
policy = []
for s in states:
    q_vals = [(a, q_value(s, a, V)) for a in actions if (s, a) in R or (s, a) in P]
    if q_vals:
        best_a = max(q_vals, key=lambda x: x[1])[0]
    else:
        best_a = "none"
    policy.append({"state": s, "action": best_a})

Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [63]:
# YOUR CHANGES HERE

policy_df = pd.DataFrame(policy)
policy_df.to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)

policy_df.head()

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,do-nothing
4,symptoms-1,drink-oj


Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [64]:
# YOUR CHANGES HERE

discomfort_df = pd.DataFrame({"state": states, "expected_discomfort": [-V[s] for s in states]})
discomfort_df.to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)
discomfort_df.head()

,state,expected_discomfort
0,exposed-1,0.75
1,exposed-2,1.50
2,exposed-3,3.00
3,recovered,-0.00
4,symptoms-1,6.00


Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [65]:
# YOUR CHANGES HERE

state_action_pairs = transitions[["state", "action"]].drop_duplicates()

sick_states = [s for s in state_action_pairs["state"].unique() if "recovered" not in s]

# Assign -1 for sick states, 0 for recovered
def assign_reward(state):
    return -1.0 if state in sick_states else 0.0

state_action_pairs["reward"] = state_action_pairs["state"].apply(assign_reward)

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [66]:
# YOUR CHANGES HERE

state_action_pairs.to_csv("duration-rewards.tsv", sep="\t", index=False)

state_action_pairs.head()

,state,action,reward
0,exposed-1,do-nothing,-1.0
2,exposed-2,do-nothing,-1.0
4,exposed-3,do-nothing,-1.0
6,symptoms-1,do-nothing,-1.0
8,symptoms-2,do-nothing,-1.0


Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [67]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
duration_rewards = pd.read_csv("duration-rewards.tsv", sep="\t")

states = sorted(set(transitions["state"]) | set(transitions["next_state"]) | set(duration_rewards["state"]))
actions = sorted(set(transitions["action"]) | set(duration_rewards["action"]))

P = {}
for s in states:
    for a in actions:
        mask = (transitions["state"] == s) & (transitions["action"] == a)
        if mask.any():
            P[(s, a)] = list(zip(transitions.loc[mask, "next_state"], transitions.loc[mask, "probability"]))

R = {(row["state"], row["action"]): float(row["reward"]) for _, row in duration_rewards.iterrows()}

gamma = 1
threshold = 1e-8
V = {s: 0.0 for s in states}

def q_value(state, action, V_current):
    r = R.get((state, action), 0.0)
    future = 0.0
    if (state, action) in P:
        for s2, p in P[(state, action)]:
            future += p * V_current[s2]
    return r + gamma * future

while True:
    delta = 0.0
    V_new = {}
    for s in states:
        cand = [a for a in actions if (s, a) in R or (s, a) in P]
        if cand:
            q_vals = [q_value(s, a, V) for a in cand]
            V_new[s] = max(q_vals)  
        else:
            V_new[s] = V[s]
        delta = max(delta, abs(V_new[s] - V[s]))
    V = V_new
    if delta < threshold:
        break

policy_rows = []
for s in states:
    cand = [a for a in actions if (s, a) in R or (s, a) in P]
    if cand:
        q_pairs = [(a, q_value(s, a, V)) for a in cand]
        best_a = max(q_pairs, key=lambda x: x[1])[0]
    else:
        best_a = "none"
    policy_rows.append({"state": s, "action": best_a})

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [68]:
# YOUR CHANGES HERE

policy_df = pd.DataFrame(policy_rows)
policy_df.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)

policy_df.head()

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,do-nothing
4,symptoms-1,do-nothing


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [69]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
duration_rewards = pd.read_csv("duration-rewards.tsv", sep="\t")
policy_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")
policy = dict(zip(policy_df["state"], policy_df["action"]))

states = sorted(set(transitions["state"]) | set(transitions["next_state"]) | set(duration_rewards["state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

P_pi = np.zeros((n, n))
r = np.zeros(n)

grouped = transitions.groupby(["state", "action"])

R = {(row["state"], row["action"]): float(row["reward"]) for _, row in duration_rewards.iterrows()}

for s in states:
    a = policy.get(s, None)
    i = idx[s]
    if a is not None:
        r[i] = -1.0 if any(v < 0 for (ss, aa), v in R.items() if ss == s) else 0.0
        
        if (s, a) in grouped.groups:
            sub = grouped.get_group((s, a))
            for _, row in sub.iterrows():
                j = idx[row["next_state"]]
                P_pi[i, j] += float(row["probability"])
    else:
        r[i] = 0.0

I = np.eye(n)
try:
    V = np.linalg.solve(I - P_pi, r)
except np.linalg.LinAlgError:
    gamma = 1
    V_iter = np.zeros(n)
    for _ in range(100000):
        V_new = r + gamma * P_pi.dot(V_iter)
        if np.max(np.abs(V_new - V_iter)) < 1e-10:
            break
        V_iter = V_new
    V = V_iter

expected_sick_days = -V  

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [70]:
# YOUR CHANGES HERE

out_df = pd.DataFrame({"state": states, "expected_sick_days": expected_sick_days})
out_df.to_csv("minimum-duration-days.tsv", sep="\t", index=False)

out_df.head()

,state,expected_sick_days
0,exposed-1,3.0
1,exposed-2,4.0
2,exposed-3,6.0
3,recovered,-0.0
4,symptoms-1,10.0


Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [75]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

policy_speed_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")   
policy_discomf_df = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t") 

policy_speed = dict(zip(policy_speed_df["state"], policy_speed_df["action"]))
policy_discomf = dict(zip(policy_discomf_df["state"], policy_discomf_df["action"]))

states = sorted(set(transitions["state"]) | set(transitions["next_state"]) | set(rewards["state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

grouped = transitions.groupby(["state", "action"])

R = {(row["state"], row["action"]): float(row["reward"]) for _, row in rewards.iterrows()}

def evaluate_policy(policy, gamma=1, threshold=1e-8):
    V = np.zeros(n)
    while True:
        delta = 0.0
        V_new = np.copy(V)
        for s in states:
            a = policy.get(s, None)
            i = idx[s]
            if a is None:
                continue
            r_sa = R.get((s, a), 0.0)
            if (s, a) in grouped.groups:
                sub = grouped.get_group((s, a))
                future = np.sum(sub["probability"].to_numpy() * np.array([V[idx[ns]] for ns in sub["next_state"].to_numpy()]))
            else:
                future = 0.0
            V_new[i] = r_sa + gamma * future
            delta = max(delta, abs(V_new[i] - V[i]))
        V = V_new
        if delta < threshold:
            break
    return -V

speed_discomfort = evaluate_policy(policy_speed)          
min_discomfort = evaluate_policy(policy_discomf)

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [76]:
# YOUR CHANGES HERE

out_df = pd.DataFrame({
    "state": states,
    "speed_discomfort": speed_discomfort,
    "minimize_discomfort": min_discomfort
})
out_df.to_csv("policy-comparison.tsv", sep="\t", index=False)
out_df.head()

,state,speed_discomfort,minimize_discomfort
0,exposed-1,0.833333,0.75
1,exposed-2,1.666667,1.50
2,exposed-3,3.333333,3.00
3,recovered,-0.000000,-0.00
4,symptoms-1,6.666667,6.00


Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

None